In [1]:
import threading
from MasterTradePy.api import MasterTradeAPI
from MasterTradePy.model import *
from MasterTradePy.constant import PriceType, OrderType, TradingSession, Side, TradingUnit, RCode

In [2]:
username = 'your_username'
password = 'your_password'
stock_id_list = ['1216', '1582', '2108', '2373', '6277', '9911', '9943']

In [3]:
event = threading.Event()

In [4]:
class ConcreteMarketTrader(MarketTrader):
    def OnNewOrderReply(self, data) -> None:
        print(data)

    def OnChangeReply(self, data) -> None:
        print(data)

    def OnCancelReply(self, data) -> None:
        print(data)

    def OnReport(self, data) -> None:
        if type(data) is ReportOrder:
            if data.order.tableName == "ORD:TwsOrd":
                print(f'回報資料: 委託書號={data.order.ordNo}, 股票代號={data.order.symbol}, 委託股數={data.orgOrder.qty}, 成交股數={data.order.cumQty}, 訊息={data.lastMessage}, 狀態={data.order.status}')
            elif data.order.tableName == "RPT:TwsDeal":
                print(f'回報資料: 委託書號={data.order.ordNo}, 股票代號={data.order.symbol}, 成交價格={data.order.dealPri}, 成交股數={data.order.cumQty}, 剩餘股數={data.order.leavesQty} 訊息={data.lastMessage}, 狀態={data.order.status}')
            elif data.order.tableName == "RPT:TwsNew":
                print(f'回報資料: 委託書號={data.order.ordNo}, 股票代號={data.order.symbol}, 委託價格={data.orgOrder.price}, 委託股數={data.orgOrder.qty}, 訊息={data.lastMessage}, 狀態={data.order.status}')
            else:
                print(f'回報資料: 委託書號={data.order.ordNo}, 股票代號={data.order.symbol}, 委託價格={data.orgOrder.price}, 委託股數={data.orgOrder.qty}, 成交股數={data.order.cumQty}, 訊息={data.lastMessage}, 狀態={data.order.status}')
    
    def OnReqResult(self, workID: str, data) -> None:
        pass

    def OnSystemEvent(self, data: SystemEvent) -> None:
        print(f'OnSystemEvent{data}')
    
    def OnAnnouncementEvent(self, data)->None:
        print(f'OnAnnouncementEvent:{data}')
                  
    def OnError(self, data):
        print(data)

def OnDigitalSSOEvent(aIsOK, aMsg):
    print(f'OnDigitalSSOEvent: {aIsOK} {aMsg}')

def OnTAConnStuEvent(aIsOK):
    print(f'OnTAConnStuEvent: {aIsOK}')
    if aIsOK:
        event.set()


In [ ]:
def execute_order(api, stock_id_list):
    account = 'your_account'
    price = ''  # 空白表示市價下單
    qtr = '1'  # 1張請輸入1
    orderType = OrderType.ROD

    if not price:
        priceType = PriceType.MKT
    else:
        priceType = PriceType.LMT

    for stock_id in stock_id_list:
        symbol = stock_id
        order = Order(tradingSession=TradingSession.NORMAL,
                      side=Side.Buy,
                      symbol=symbol,
                      priceType=priceType,
                      price=price,
                      tradingUnit=TradingUnit.COMMON, 
                      qty=qty,
                      orderType=orderType,
                      tradingAccount=account,
                      userDef='')
        rcode = api.NewOrder(order)
        if rcode == RCode.OK:
            print(f'已送出委託: {symbol}')
        else:
            print(f'下單失敗! 股票代號: {symbol}，請再次執行程式，依據回報資料修正輸入')

In [6]:
def main():
    trader = ConcreteMarketTrader()
    api = MasterTradeAPI(trader)
    api.SetConnectionHost('solace140.masterlink.com.tw:55555')
    
    # 登入交易主機
    rc = api.Login(username, password, True, True, True)
    if rc == RCode.OK:
        print('交易主機連線成功，進行雙因子認證')
        
        # 取得帳戶並進行驗證
        accounts = [x[4:] for x in api.accounts]
        rcc = api.CheckAccs(tradingAccounts=accounts)
        if rcc == RCode.OK:
            print('驗證已通過，可執行 API 交易功能')

    execute_order(api, stock_id_list)

    input("Press Enter to finish...\n")

main()


連線競賽ID:C121695520VA
OnAnnouncementEvent:證券交易驗證公告內文 : 證券登入公告
期貨交易驗證公告內文 : 期貨登入公告
連線競賽主機!!!

交易主機連線成功，進行雙因子認證
驗證已通過，可執行 API 交易功能
回報資料: 委託書號=Y0001, 股票代號=3515, 委託價格=, 委託股數=5000, 訊息=新單 5000 股OK!, 狀態=101)委託已接受(交易所已接受)
已送出委託: 3515
已送出委託: 2610
回報資料: 委託書號=Y0002, 股票代號=2610, 委託價格=, 委託股數=5000, 訊息=新單 5000 股OK!, 狀態=101)委託已接受(交易所已接受)
已送出委託: 2344
回報資料: 委託書號=Y0003, 股票代號=2344, 委託價格=, 委託股數=5000, 訊息=新單 5000 股OK!, 狀態=101)委託已接受(交易所已接受)
已送出委託: 2317
回報資料: 委託書號=Y0004, 股票代號=2317, 委託價格=, 委託股數=5000, 訊息=新單 5000 股OK!, 狀態=101)委託已接受(交易所已接受)
已送出委託: 2460
回報資料: 委託書號=Y0005, 股票代號=2460, 委託價格=, 委託股數=5000, 訊息=新單 5000 股OK!, 狀態=101)委託已接受(交易所已接受)
